In [2]:
import wrds
import pandas as pd
import numpy as np

# Connexion WRDS
db = wrds.Connection()

# Liste de 50 tickers S&P 500
tickers_50 = [
    'AAPL','MSFT','AMZN','GOOGL','META','BRK.B','JNJ','JPM','V','PG',
    'XOM','NVDA','HD','MA','BAC','UNH','DIS','PFE','VZ','ADBE',
    'NFLX','INTC','KO','T','WMT','CSCO','PEP','CVX','MRK','ORCL',
    'ABT','CRM','COST','ACN','AVGO','MDT','MCD','AMGN','TMO','ABBV',
    'LIN','PM','TXN','NEE','HON','UNP','UPS','LOW','SBUX','IBM'
]

tickers_sql = "'" + "', '".join(tickers_50) + "'"

query = f"""
    SELECT
        s.tic,
        s.datadate,
        s.eps,
        s.prccd,
        s.gvkey
    FROM comp_na_daily_all.secd AS s
    WHERE s.datadate BETWEEN '2010-01-01'::date AND '2025-11-16'::date
      AND s.tic IN ({tickers_sql})
"""

df = db.raw_sql(query)

df["datadate"] = pd.to_datetime(df["datadate"])
df = df.sort_values(["tic", "datadate"])

# --- Conversion en numérique ---
df["eps"] = pd.to_numeric(df["eps"], errors="coerce").astype(float)
df["prccd"] = pd.to_numeric(df["prccd"], errors="coerce").astype(float)

# --- P/E instantané ---
df["pe_ratio"] = np.where(
    (df["eps"].notna()) & (df["eps"] > 0),
    df["prccd"] / df["eps"],
    np.nan
)

# --- P/E glissant ---
WINDOW = 252  # ~1 an

df["pe_ratio_roll_252"] = (
    df.groupby("tic")["pe_ratio"]
      .transform(lambda x: x.rolling(WINDOW, min_periods=60).mean())
)

# --- Export Excel ---
output_file = "sp500_50_tickers_pe_glissant.xlsx"
df.to_excel(output_file, index=False)

print("Fichier Excel créé :", output_file)


WRDS recommends setting up a .pgpass file.
pgpass file created at C:\Users\berge\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Fichier Excel créé : sp500_50_tickers_pe_glissant.xlsx
